In [1]:
import sys
sys.path.append("..")
from ctdi_treatment.env_setup_start import *
import time
license_keys = set_envvars('/home/ubuntu/hasham/jsl_keys.json')
spark = start_sparknlp(license_keys)

Desired SparkNLP Version: 3.2.1
Desired SparkNLP-JSL Version: 3.2.0
Real Spark NLP Version : 3.2.1
Real Spark NLP_JSL Version : 3.2.0


## File loading and pre-processing

In [2]:
import pandas as pd
alldf = pd.read_csv("../data/raw_aac.csv")
print (alldf.shape)
print (alldf.columns)
alldf.head()


(45, 12)
Index(['design_group_id', 'nct_id', 'group_type', 'title', 'arm_desc',
       'intervention_id', 'intervention_type', 'name', 'intervention_desc',
       'arm_title_desc', 'intervention_name_desc',
       'intervention_type_name_desc'],
      dtype='object')


design_group_id       nct_id          group_type  title  \
0         13299803  NCT00000435  Placebo Comparator      A   
1         13299804  NCT00000435   Active Comparator      B   
2         13690845  NCT00001337        Experimental  Arm A   
3         13690845  NCT00001337        Experimental  Arm A   
4         13732309  NCT00001563        Experimental      1   

                                            arm_desc  intervention_id  \
0  Subjects randomized to arm A received 25mg/day...         12910066   
1  Subjects randomized to Arm B received 25mg/day...         12910065   
2      EPOCH + Rituximab every 3 weeks for 6 cycles.         13274570   
3      EPOCH + Rituximab every 3 weeks for 6 cycles.         13274571   
4            EPOCH-R every 3 weeks for up to 6 cycle         13311178   

  intervention_type          name  \
0              Drug  None-placebo   
1              Drug  dnaJ peptide   
2              Drug         EPOCH   
3        Biological     Rituximab   
4        Biological    Filgrastim   

                                   intervention_desc  \
0  placebo was taken in pill form at 25mg/day for...   
1  dnaJP1 was taken in pill form at 25mg/day for ...   
2  Combination chemotherapy given with Rituximab ...   
3  Rituximab given on Day 1 of combination chemot...   
4  Filgrastim after EPOCH-R from Day 6 for 10 day...   

                                      arm_title_desc  \
0  A\n\nSubjects randomized to arm A received 25m...   
1  B\n\nSubjects randomized to Arm B received 25m...   
2  Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...   
3  Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...   
4       1\n\nEPOCH-R every 3 weeks for up to 6 cycle   

                              intervention_name_desc  \
0  None-placebo: placebo was taken in pill form a...   
1  dnaJ peptide: dnaJP1 was taken in pill form at...   
2  EPOCH: Combination chemotherapy given with Rit...   
3  Rituximab: Rituximab given on Day 1 of combina...   
4  Filgrastim: Filgrastim after EPOCH-R from Day ...   

                         intervention_type_name_desc  
0  Drug\nNone-placebo: placebo was taken in pill ...  
1  Drug\ndnaJ peptide: dnaJP1 was taken in pill f...  
2  Drug\nEPOCH: Combination chemotherapy given wi...  
3  Biological\nRituximab: Rituximab given on Day ...  
4  Biological\nFilgrastim: Filgrastim after EPOCH...

In [3]:
name_sep = ": "
sent_sep = "."
tit_sep = "\n\n"

# Join title and arm description
alldf["arm_title_desc"] = alldf["title"]+tit_sep+alldf["arm_desc"]
# Join name and intervention desc
alldf["intervention_name_desc"] = alldf["name"]+name_sep+alldf["intervention_desc"]+sent_sep
# Join intervention type and name and intervention desc
#alldf["intervention_type_name_desc"] = alldf["intervention_type"]+tit_sep+alldf["intervention_name_desc"]
alldf.head()

design_group_id       nct_id          group_type  title  \
0         13299803  NCT00000435  Placebo Comparator      A   
1         13299804  NCT00000435   Active Comparator      B   
2         13690845  NCT00001337        Experimental  Arm A   
3         13690845  NCT00001337        Experimental  Arm A   
4         13732309  NCT00001563        Experimental      1   

                                            arm_desc  intervention_id  \
0  Subjects randomized to arm A received 25mg/day...         12910066   
1  Subjects randomized to Arm B received 25mg/day...         12910065   
2      EPOCH + Rituximab every 3 weeks for 6 cycles.         13274570   
3      EPOCH + Rituximab every 3 weeks for 6 cycles.         13274571   
4            EPOCH-R every 3 weeks for up to 6 cycle         13311178   

  intervention_type          name  \
0              Drug  None-placebo   
1              Drug  dnaJ peptide   
2              Drug         EPOCH   
3        Biological     Rituximab   
4        Biological    Filgrastim   

                                   intervention_desc  \
0  placebo was taken in pill form at 25mg/day for...   
1  dnaJP1 was taken in pill form at 25mg/day for ...   
2  Combination chemotherapy given with Rituximab ...   
3  Rituximab given on Day 1 of combination chemot...   
4  Filgrastim after EPOCH-R from Day 6 for 10 day...   

                                      arm_title_desc  \
0  A\n\nSubjects randomized to arm A received 25m...   
1  B\n\nSubjects randomized to Arm B received 25m...   
2  Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...   
3  Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...   
4       1\n\nEPOCH-R every 3 weeks for up to 6 cycle   

                              intervention_name_desc  \
0  None-placebo: placebo was taken in pill form a...   
1  dnaJ peptide: dnaJP1 was taken in pill form at...   
2  EPOCH: Combination chemotherapy given with Rit...   
3  Rituximab: Rituximab given on Day 1 of combina...   
4  Filgrastim: Filgrastim after EPOCH-R from Day ...   

                         intervention_type_name_desc  
0  Drug\nNone-placebo: placebo was taken in pill ...  
1  Drug\ndnaJ peptide: dnaJP1 was taken in pill f...  
2  Drug\nEPOCH: Combination chemotherapy given wi...  
3  Biological\nRituximab: Rituximab given on Day ...  
4  Biological\nFilgrastim: Filgrastim after EPOCH...

In [5]:
annotation_df = alldf.groupby(["nct_id","design_group_id","group_type","title"]).agg(
    intervention_name_desc=("intervention_name_desc", tit_sep.join),
    arm_title_desc = ('arm_title_desc', tit_sep.join),
    intervention_type = ( 'intervention_type', '|'.join),
#     group_type = ( 'group_type' , tit_sep.join ),
#     title = (  'title', tit_sep.join )
).reset_index()
annotation_df.shape

(13, 7)

## Place 2 important files in the ctdi_treatment folder
- treatment_pipeline.py (Please update older version of the file with the one provided with this script)
- resolver_pipeline.py


In [4]:
from ctdi_treatment.treatment_pipeline import TreatmentPipeline

main_pipeline = TreatmentPipeline(spark)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[OK!]
ner_clinical_large download started this may take some time.
Approximate size to download 13.9 MB
[OK!]
ner_jsl download started this may take some time.
Approximate size to download 14.5 MB
[OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[OK!]
Main Pipeline Created...
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_rxnorm_disposition download started this may take some 

## Now run the dataframe using treatment.py file - Please update the file with the new one provided with this script

In [6]:

from treatment import treatment_extraction_full


annotation_df['arm'] = annotation_df['title']

annotation_df['intervention_data'] = annotation_df['intervention_name_desc']

res = treatment_extraction_full(main_pipeline, annotation_df, custom_col_name=None)

res.head()

Running full solution smartly...
Ner Pipeline Executed...
Preparing Data for Resolvers...
Data Prepared for Resolvers...
Executing Resolver Pipeline...
Resolver Pipeline Executed...
Preparing Full Result...
Result Mapped and Ready...


nct_id  design_group_id          group_type  \
0  NCT00000435         13299803  Placebo Comparator   
1  NCT00000435         13299804   Active Comparator   
2  NCT00001337         13690845        Experimental   
3  NCT00001563         13732309        Experimental   
4  NCT00001575         13299883        Experimental   

                                               title  \
0                                                  A   
1                                                  B   
2                                              Arm A   
3                                                  1   
4  Anti-Tac yttrium 90-labeled humanized anti-Tac...   

                              intervention_name_desc  \
0  None-placebo: placebo was taken in pill form a...   
1  dnaJ peptide: dnaJP1 was taken in pill form at...   
2  EPOCH: Combination chemotherapy given with Rit...   
3  Filgrastim: Filgrastim after EPOCH-R from Day ...   
4  Y-90 Humanized Anti-Tac: 10 mCi (if a bone mar...   

                                      arm_title_desc  \
0  A\n\nSubjects randomized to arm A received 25m...   
1  B\n\nSubjects randomized to Arm B received 25m...   
2  Arm A\n\nEPOCH + Rituximab every 3 weeks for 6...   
3  1\n\nEPOCH-R every 3 weeks for up to 6 cycle\n...   
4  Anti-Tac yttrium 90-labeled humanized anti-Tac...   

            intervention_type  \
0                        Drug   
1                        Drug   
2             Drug|Biological   
3  Biological|Biological|Drug   
4             Biological|Drug   

                                                 arm  \
0                                                  A   
1                                                  B   
2                                              Arm A   
3                                                  1   
4  Anti-Tac yttrium 90-labeled humanized anti-Tac...   

                                   intervention_data  \
0  None-placebo: placebo was taken in pill form a...   
1  dnaJ peptide: dnaJP1 was taken in pill form at...   
2  EPOCH: Combination chemotherapy given with Rit...   
3  Filgrastim: Filgrastim after EPOCH-R from Day ...   
4  Y-90 Humanized Anti-Tac: 10 mCi (if a bone mar...   

                                 treatment_extracted  \
0  [{'raw_sentence': 'None-placebo', 'raw_drug': ...   
1  [{'raw_sentence': 'dnaJP1 was taken in pill fo...   
2  [{'raw_sentence': 'Combination chemotherapy gi...   
3  [{'raw_sentence': 'Filgrastim', 'raw_drug': 'F...   
4  [{'raw_sentence': 'Y-90 Humanized Anti-Tac', '...   

                                   arm_reconstructed  \
0                                  neverpain+placebo   
1                                                      
2                                    rituximab+epoch   
3                         rituximab+epoch+filgrastim   
4  yttrium bromide+pentetate calcium trisodium+dy...   

                                        standard_arm  
0                                  neverpain+placebo  
1                                                  B  
2                                    rituximab+epoch  
3                         rituximab+epoch+filgrastim  
4  yttrium bromide+pentetate calcium trisodium+dy...